In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

In [2]:
#!unzip '/content/stacksample.zip?dl=0'

In [3]:
questions = r'D:\Work\DS_and_AI\Deep_Learning\Project\Project_1\stacksample\Questions.csv'
answers = r'D:\Work\DS_and_AI\Deep_Learning\Project\Project_1\stacksample\Answers.csv'
tags = r'D:\Work\DS_and_AI\Deep_Learning\Project\Project_1\stacksample\Tags.csv'

In [6]:
questions_data = pd.read_csv(questions,encoding ='latin 1' )
answers_data = pd.read_csv(answers,encoding ='latin 1')
tags_data = pd.read_csv(tags,encoding ='latin 1')

In [7]:
questions_data.head(n=2)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...


In [8]:
tags_data.head(2)

,Id,Tag
0,80,flex
1,80,actionscript-3


In [9]:
#grouping the tags
grouped_tags = tags_data.groupby("Tag", sort='count').size().reset_index(name='count')

In [10]:
grouped_tags

,Tag,count
0,.a,13
1,.app,14
2,.aspxauth,6
3,.bash-profile,46
4,.class-file,19
...,...,...
37029,zwoptex,3
37030,zxing,159
37031,zxspectrum,1
37032,zynq,11


In [11]:
grouped_tags.Tag.describe()

count                  37034
unique                 37034
top       grunt-contrib-sass
freq                       1
Name: Tag, dtype: object

In [12]:
#Reducing the problem to the most common tags in the dataset
num_classes = 10
#grouped_tags = tags_data.groupby("Tag").size().reset_index(name='count')
top_10_tags = grouped_tags.nlargest(num_classes,columns="count")

In [13]:
top_10_tags

,Tag,count
16121,javascript,124155
16020,java,115212
4301,c#,101186
24211,php,98808
979,android,90659
16611,jquery,78542
25712,python,64601
14180,html,58976
4311,c++,47591
15565,ios,47009


In [14]:
#filtering the top 10 tags data 
tags_data.Tag = tags_data.Tag.apply(lambda tag : tag if tag in top_10_tags.Tag.values else None)

In [15]:
tags_data

,Id,Tag
0,80,None
1,80,None
2,80,None
3,90,None
4,90,None
...,...,...
3750989,40143360,javascript
3750990,40143360,None
3750991,40143380,None
3750992,40143380,None


In [16]:
tags_data.dropna(inplace=True)

In [17]:
tags_data

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#
...,...,...
3750974,40143150,javascript
3750981,40143190,python
3750984,40143210,php
3750987,40143340,android


In [18]:
#preparing the contents of dataframe
#text = '<p dgqwifgwi> This is important </p>'
import re

def strip_html_tags(body) :
  regex = re.compile('<.*?>')
  return re.sub(regex, '', body)


In [19]:
questions_data['Body'] = questions_data.Body.apply(strip_html_tags)

In [20]:
questions_data['Body']

0          I've written a database generation script in S...
1          Are there any really good tutorials explaining...
2          Has anyone got experience creating SQL-based A...
3          This is something I've pseudo-solved many time...
4          I have a little game written in C#. It uses a ...
                                 ...                        
1264211    I am building a custom MVC project and I have ...
1264212    The API docs show that you should be able to m...
1264213    Under minifyEnabled I changed from false to tr...
1264214    I have input which I use to filter my array of...
1264215    I'm running a mocha test and I noticed my chan...
Name: Body, Length: 1264216, dtype: object

In [21]:
questions_data['Text'] = questions_data['Title']+' '+questions_data['Body']

In [22]:
questions_data['Text']

0          SQLStatement.execute() - multiple queries in o...
1          Good branching and merging tutorials for Torto...
2          ASP.NET Site Maps Has anyone got experience cr...
3          Function for creating color wheels This is som...
4          Adding scripting functionality to .NET applica...
                                 ...                        
1264211    URL routing in PHP (MVC) I am building a custo...
1264212    Bigquery.Jobs.Insert - Resumable Upload? The A...
1264213    Obfuscating code in android studio Under minif...
1264214    How to fire function after v-model change? I h...
1264215    npm run mocha test - files being cached I'm ru...
Name: Text, Length: 1264216, dtype: object

In [23]:
questions_data.shape

(1264216, 8)

In [24]:
tags_data.shape

(826739, 2)

In [25]:
questions_data[17:19]

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Text
17,1390,60.0,2008-08-04T16:33:36Z,NaN,18,"Is Windows Server 2008 ""Server Core"" appropria...",I'm setting up a dedicated SQL Server 2005 box...,"Is Windows Server 2008 ""Server Core"" appropria..."
18,1600,230.0,2008-08-04T21:27:53Z,NaN,18,What is the best way to copy a database?,"I always create a new empty database, after th...",What is the best way to copy a database? I alw...


In [26]:
tags_data.head(2)

,Id,Tag
14,260,c#
18,330,c++


In [27]:
# denormalize tables
def tags_for_question(question_id):
    return tags_data[tags_data['Id'] == questions_data].values

def add_tags_column(row):
    row['Tags'] = tags_for_question(row['Id'])
    return row


In [ ]:
#questions_data = questions_data.apply(add_tags_column, axis=1)